In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
# 读取微状态序列的函数
# 定义一个函数，接受文件路径和可选的固定长度参数
def read_microstate_sequence(file_path, fixed_length=None):
    with open(file_path, 'r') as f:
        sequence = np.loadtxt(f, dtype=int)
        if fixed_length is not None:
            sequence = sequence[:fixed_length]
            if len(sequence) < fixed_length:
                sequence = np.pad(sequence, (0, fixed_length - len(sequence)), 'constant')
    return sequence

In [ ]:
# 计算熵的函数
def H_k(x, ns, k):
    counts = np.bincount(x[-k:], minlength=4)  # 计算最后k个状态的频率
    probabilities = counts / float(k)  # 计算概率
    entropy = -np.sum(probabilities[probabilities > 0] * np.log2(probabilities[probabilities > 0]))  # 计算熵
    return entropy

In [ ]:
# 计算熵率和过剩熵
def excess_entropy_rate(x, ns, kmax):
    h_ = np.zeros(kmax)  # 初始化熵数组
    for k in range(kmax):
        h_[k] = H_k(x, ns, k + 1)

    ks = np.arange(1, kmax + 1)
    print(f"h_ values: {h_}")  # 打印 h_ 数组
    a, b = np.polyfit(ks, h_, 1)  # a = 熵率, b = 超额熵

    # 确保熵率和过剩熵是非负的
    entropy_rate = max(a, 0)
    excess_entropy = max(b, 0)
    return entropy_rate, excess_entropy

In [ ]:
# 计算熵率和过剩熵
def compute_entropy_rate_and_excess_entropy(directory, kmax=10, fixed_length=None):
    results = {}
    
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            microstate_sequence = read_microstate_sequence(file_path, fixed_length)

            ns = len(microstate_sequence)  # 计算微状态序列的数据点数量

            # 检查数据点数量
            if ns > kmax + 1:
                try:
                    entropy_rate, excess_entropy = excess_entropy_rate(microstate_sequence, ns, kmax)
                    results[filename] = {
                        'entropy_rate': entropy_rate,
                        'excess_entropy': excess_entropy
                    }
                    print(f"Processed {filename}: ns={ns}, Entropy Rate={entropy_rate}, Excess Entropy={excess_entropy}")
                except np.linalg.LinAlgError as e:
                    print(f"LinAlgError for {filename}: {e}")
                    continue
            else:
                print(f"Warning: {filename} has insufficient data points for entropy calculation.")
    
    return results

In [ ]:
# 设置随机种子
np.random.seed(123)

# 示例使用
directory = r'F:\EEGmicrostate\microstate_2_20\groupMicrostate\EO\MicorstateSeuqnce2group\all\full'
#directory = r'F:\EEGmicrostate\microstate_2_20\groupMicrostate\EO\MicorstateSeuqnce2group\all\full'
kmax = 6
fixed_length = 30000

In [ ]:
results = compute_entropy_rate_and_excess_entropy(directory, kmax, fixed_length)

In [ ]:
if results:
    for filename, metrics in results.items():
        print(f"File: {filename}, Entropy Rate: {metrics['entropy_rate']}, Excess Entropy: {metrics['excess_entropy']}")
else:
    print("No results found.")

In [ ]:
# 保存结果到 CSV 文件
if results:
    # 创建一个列表以存储结果
    formatted_results = []
    
    # 格式化每个结果
    for filename, metrics in results.items():
        formatted_results.append({
            'File': filename,
            'Entropy Rate': metrics['entropy_rate'],
            'Excess Entropy': metrics['excess_entropy']
        })
    
    # 创建 DataFrame
    df = pd.DataFrame(formatted_results)
    # 保存为 CSV 文件
    df.to_csv(os.path.join(directory, 'full_entropy_results_30000k6.csv'), index=False)

    # 输出信息
    for entry in formatted_results:
        print(f"File: {entry['File']}, Entropy Rate: {entry['Entropy Rate']}, Excess Entropy: {entry['Excess Entropy']}")
else:
    print("No results found.")